<a href="https://colab.research.google.com/github/tempiatine/hw_piatin/blob/main/Meteoanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# !pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, max, sum, avg, month

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Meteo").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("/content/sample_data/weather_data.csv", header=True, inferSchema=True)

# Выполнение SQL-запроса c топ-5 самых жарких дней
hot_df = df.select("date", "temperature").orderBy(col("temperature").desc())

# Выполнение SQL-запроса c метеостанцией с наибольшим количеством осадков за последний год
# Находим максимальный год
max_year = df.agg(max(year(col("date")))).collect()[0][0]
# Фильтруем DataFrame
precip_df = df.select("station_id", "precipitation").where(year(col("date")) == max_year).groupBy('station_id').agg(sum('precipitation')).orderBy(sum('precipitation').desc())

# Выполнение SQL-запроса cо средней температурой по месяцам
avg_temp_df = df.select('date', 'temperature').groupBy(month(col("date"))).agg(avg('temperature')).orderBy(month(col("date")))

# Показ результатов
hot_df.show(5)
precip_df.show(1)
avg_temp_df.show()

# Остановка SparkSession
spark.stop()

+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+
only showing top 5 rows

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302626767898|
+----------+------------------+
only showing top 1 row

+-----------+------------------+
|month(date)|  avg(temperature)|
+-----------+------------------+
|          1|11.356518462550754|
|          2| 9.067229891101926|
|          3| 7.244080205633994|
|          4|12.024529009744693|
|          5| 9.902883346912718|
|          6|13.421092297254138|
|          7|6.1857183016954576|
|          8|  10.9678002814186|
|          9| 9.596744236573942|
|         10|  9.09884344821895|
|         11| 7.265889994697494|
|         12|11.218592100674337|
+-------